# Подготовка

In [ ]:
import os
os.chdir("../../..")

In [ ]:
import jax
jax.devices()

In [ ]:
from scripts.nj.neurosci import *
import scripts.nj.graph_to_arrays as ga
import pandas as pd

In [ ]:
DIR = "Ilya/trash/holy_data/jax/"
res = ga.SimulationContextJax.load_context_from_cache(DIR, initial_node_values={'cable':1.0})

In [ ]:
len(res['mapping']['cable'])

In [ ]:
local_indeces_mapping = res['mapping']

In [ ]:
# Обойдемся пока без этого
#path_to_metadata = "Ilya/trash/Neurons_Metadata/Metadata/Metadata_Nodes(manual).csv"
#metadata = pd.read_csv(path_to_metadata)
#metadata = metadata.fillna(10.0) # 10.0 as basic radius
#metadata['new_index'] = metadata.apply(lambda row:local_indeces_mapping['cable'].get(str(row['node_id'])), axis = 1)
#metadata = metadata.dropna(subset=['new_index'])
#metadata = metadata.set_index('new_index').sort_index()
#metadata

# Определение симуляции

In [ ]:
def get_my_pipeline(constants, dt = 0.1):
    cable_m = res['edges_cable_to_cable'].T
    pre_syn = res['edges_cable_to_alpha'].T
    post_syn = res['edges_cable_to_alpha'].T

    HH = get_HH_pipeline(**constants) # получаем функцию для HH
    cable = laplace_at_graph_symetric(cable_m, 'V')#, scaling = R) # получаем функцию для динамики кабелей
    alphaP = get_alpha_synapce_pipeline(pre_syn, post_syn, **constants)
    
    @jax.jit
    def state_transformed(state):
        s, ds = to_diff(state) # создает ds той же формы что и state, но заполненный нулями
        s, ds = alphaP(s, ds)
        s, ds = HH(s, ds) # вставляет HH каналы
        s, ds = cable(s, ds) # соединяет сегменты
        ds['V'] += ds['V'].at[0].add(10.0*(s['time'] > 20.0))
        return s, ds

    integrate = get_runge_kutta_step(state_transformed, dt) # получаем функцию для интегрирования
    @jax.jit
    def my_pipeline(state):
        s = integrate(state) # интегрируем
        return s
    return my_pipeline


In [ ]:
num_nodes_hh = res['num_nodes']['cable']
num_synapces = res['num_nodes']['alpha']
total_nodes = num_nodes_hh + num_synapces

initials = {
    "V":jnp.ones((num_nodes_hh, ), jnp.float32)*-65.0,
    "m":jnp.ones((num_nodes_hh, ), jnp.float32)*0.0220,
    'n':jnp.ones((num_nodes_hh, ), jnp.float32)*0.0773,
    'h':jnp.ones((num_nodes_hh, ), jnp.float32)*0.9840,
    'alpha':jnp.ones((num_nodes_hh, 2), jnp.float32)*0.1,
    "time":0.0
}

consts = {
    "C": jnp.ones((num_nodes_hh, ), jnp.float32),# Емкость мембраны (мкФ/см^2)
    "ENa": 50.0,   # Равновесный потенциал Na+ (мВ)
    "EK": -77.0,   # Равновесный потенциал K+ (мВ)
    "EL": -54.4,   # Равновесный потенциал утечки (мВ)
    "gNa": 120.0,  # Максимальная проводимость Na+ (мСм/см^2)
    "gK": 36.0,    # Максимальная проводимость K+ (мСм/см^2)
    "gL": 0.3,     # Проводимость утечки (мСм/см^2),
    "tau":1.0,
    'E_rev':1.0,
    'V_m':jnp.ones((num_nodes_hh, ), jnp.float32),
    'alpha_syn_detector_treshold':40.0,
    'synaptic_weights':0.01,
    'G_max':1.0
}


In [ ]:
my_pipeline = get_my_pipeline(consts, 0.01)

In [ ]:
jsim = simulation(initials, my_pipeline, 100)
H = jsim.run(10)

In [ ]:
#jnp.savez("output.npz", H) <- сохранние результатов, единственная проблема. Слишком долго